# Capstone Project Part 3 
## Using pretrained model from Hugging face

#1. Install related packages

In [ ]:
!pip install --upgrade urllib3==1.25.4

     |████████████████████████████████| 133kB 9.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
%%bash
pip install tqdm boto3 requests regex sentencepiece sacremoses

In [ ]:
! pip3 install torch torchvision


In [ ]:
! pip install transformers

     |████████████████████████████████| 2.3MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 68.5MB/s 


#2. import libs for translations

In [ ]:
import torch
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#3.Translation from English to Chinese

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh")

In [ ]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [ ]:
translation = pipeline("translation_en_to_zh", model=model, tokenizer=tokenizer)

In [ ]:
translation

In [ ]:
text = "I like to study Data Science and Machine Learning"
translated_text = translation(text, max_length=40)[0]['translation_text']
print(translated_text)

我喜欢学习数据科学和机器学习


# 4.Translation from Chinese to English

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

In [ ]:
translationZh_En = pipeline("translation_zh_to_en", model=model, tokenizer=tokenizer)

In [ ]:
textZh = '今天我要跟大家讲述 过去30年的建筑史。'
translated_text = translationZh_En(textZh, max_length=40)[0]['translation_text']
print(translated_text)

Today I'm going to tell you about the history of architecture over the past 30 years.


# 5. Build an explainer and visualisation 

In [ ]:
! pip install shap

     |████████████████████████████████| 358kB 8.2MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491620 sha256=989af80d0d7f1cf18a64a035c8f89bc42cbcc3762ff3e0d46af56a2a30ae9818
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
#import libs
import numpy as np
import shap


In [ ]:
# -*- coding: utf-8 -*-
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

data = ['今天我要跟大家讲述 过去30年的建筑史。']

In [ ]:
#build an explainer by passing the model that we want to explain and 
#the tokennizer we want to use to break up the input strings.
explainer = shap.Explainer(model,tokenizer)


In [ ]:
#explainers are callable, just like models
shap_values= explainer(data,fixed_context=1)
shap.plots.text(shap_values)

,今天,我要,跟,大家,讲述,过去,30,年,的,建筑,史,。,
Today,9.305,-0.224,-0.096,-0.096,-0.143,0.055,-0.094,0.212,-0.153,-0.053,0.023,0.09,0.089
I,1.153,2.53,0.009,-0.158,-0.391,-0.089,-0.037,-0.06,-0.084,-0.096,-0.175,0.578,0.049
',0.649,0.149,0.399,-0.487,-0.035,-0.106,0.043,-0.122,-0.008,-0.035,-0.117,0.074,0.017
m,0.3,0.369,0.272,-0.613,-0.771,-0.08,0.054,-0.02,0.073,-0.075,0.015,0.191,-0.02
going,0.783,1.135,-0.086,0.502,0.532,0.069,-0.203,-0.228,0.028,-0.022,0.109,0.208,-0.035
to,-0.23,0.218,0.735,-0.09,0.605,0.016,0.003,0.04,0.044,0.032,0.034,0.072,0.019
tell,0.226,0.16,0.562,0.724,3.213,-0.209,-0.049,0.035,0.063,-0.214,0.245,-0.294,-0.039
you,-0.042,-0.111,-0.089,0.397,1.187,-0.244,-0.047,-0.009,0.107,0.207,0.326,0.165,-0.075
about,0.303,0.055,0.394,0.006,1.615,-0.563,0.006,-0.025,0.187,1.037,0.183,0.526,0.01
the,0.374,0.22,0.006,0.162,0.089,1.723,-0.033,-0.402,-0.078,0.455,0.028,-0.099,-0.098
